In [1]:
import os
import shutil
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:

def copy_files_recursive(source_folder, destination_folder):
    print('test')
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            print(file)
            source_path = os.path.join(root, file)
            destination_path = os.path.join(destination_folder, os.path.relpath(source_path, source_folder))

            # Create destination directories if they don't exist
            os.makedirs(os.path.dirname(destination_path), exist_ok=True)

            shutil.copyfile(source_path, destination_path)

# source directory where your all data is present
source_folder  = r'/content/drive/MyDrive/colab_notebooks/hackathon/benin_hackathon/data'


# do not change this path
destination_folder = '/content/data'
copy_files_recursive(source_folder, destination_folder)


test
cleaned_test_data_yor_eng_300.csv
cleaned_train_data_yor_eng_1000.csv


In [3]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split

In [4]:
Clean = False
Train = True


In [5]:
df_train = pd.read_csv('data/cleaned_train_data_yor_eng_1000.csv')


In [6]:
df_train.head()


,eng,yor
0,Construct romantic evenings with a view of und...,Kọ awọn irọlẹ ifẹ pẹlu wiwo labẹ ọrun irawọ
1,Create luxury lifestyles in the heart of refle...,Ṣẹda awọn igbesi aye igbadun ni ọkan ti afihan...
2,Experience futuristic cities against the backd...,Ni iriri awọn ilu ọjọ iwaju lodi si ẹhin ti la...
3,Capture city life in the heart of bathed in mo...,Yaworan ilu aye ni okan ti wẹ ninu oṣupa
4,Picture mythological stories surrounded by ech...,Aworan awọn itan itan ayeraye ti yika nipasẹ i...


In [7]:


# Check if there are any empty values in the DataFrame
if df_train.isna().any().any():
    print("The DataFrame contains empty values.")
else:
    print("The DataFrame does not contain empty values.")


The DataFrame does not contain empty values.


In [8]:
display(df_train)

,eng,yor
0,Construct romantic evenings with a view of und...,Kọ awọn irọlẹ ifẹ pẹlu wiwo labẹ ọrun irawọ
1,Create luxury lifestyles in the heart of refle...,Ṣẹda awọn igbesi aye igbadun ni ọkan ti afihan...
2,Experience futuristic cities against the backd...,Ni iriri awọn ilu ọjọ iwaju lodi si ẹhin ti la...
3,Capture city life in the heart of bathed in mo...,Yaworan ilu aye ni okan ti wẹ ninu oṣupa
4,Picture mythological stories surrounded by ech...,Aworan awọn itan itan ayeraye ti yika nipasẹ i...
...,...,...
983,Explore ancient ruins showcasing in serene iso...,Ṣawari awọn ahoro atijọ ti n ṣafihan ni ipinya...
984,Paint wildlife encounters at the edge of glowi...,Kun awọn alabapade ẹranko igbẹ ni eti didan pẹ...
985,Capture classical portraits overlooking alive ...,Yaworan kilasika sisunmu gbojufo laaye pẹlu awọ
986,Imagine desert sunsets on a journey through in...,Fojú inú wo bí ìwọ̀ oòrùn aṣálẹ̀ ṣe ń lọ síbi ...


In [9]:
print(988/32)
print(len(df_train))


30.875
988


In [ ]:
# i am not applying clean
if Clean:
    # converting every letter to lower case
    df_train['Yoruba'] = df_train['Yoruba'].apply(lambda x: str(x).lower())
    df_train['English'] = df_train['English'].apply(lambda x: str(x).lower())
    # removing apostrophe from the sentences
    df_train['Yoruba'] = df_train['Yoruba'].apply(lambda x: re.sub("'","",x))
    df_train['English'] = df_train['English'].apply(lambda x: re.sub("'","",x))
    exclude = set(string.punctuation)
    # removing all the punctuations
    df_train['Yoruba'] = df_train['Yoruba'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    df_train['English'] = df_train['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    # removing digits from the sentences
    digit = str.maketrans('','',string.digits)
    df_train['Yoruba'] = df_train['Yoruba'].apply(lambda x: x.translate(digit))
    df_train['English'] = df_train['English'].apply(lambda x: x.translate(digit))

In [10]:
# using pretrained model and then finetunig it on our dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en").to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [11]:
optimizer = torch.optim.AdamW(model.parameters(),lr=0.0001)


In [12]:
def model_train():
    model.train()
    losses = 0
    X = df_train['yor']
    y = df_train['eng']
    max_epochs = 27
    n_batches = 32
    for epoch in tqdm(range(max_epochs)):
        for i in tqdm(range(len(df_train)//n_batches)):
            # making batches
            local_X, local_y = X[i*n_batches:(i+1)*n_batches,], y[i*n_batches:(i+1)*n_batches,]
            # preparing the data according to the model input
            batch = tokenizer.prepare_seq2seq_batch(list(local_X),list(local_y),return_tensors='pt').to('cuda')
            output = model(**batch)
            # loss can be taken directly from the model output
            loss = output.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses = losses+loss
    average = losses/len(df_train)
    print('Loss: ' + str(average) )

    return model

In [13]:
if Train:
    model = model_train()

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is depr

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Loss: tensor(0.1187, device='cuda:0', grad_fn=<DivBackward0>)


In [14]:


model_save_path = '/content/drive/MyDrive/colab_notebooks/hackathon/benin_hackathon/model_2'
# model.save_pretrained(model_save_path)
# tokenizer.save_pretrained(model_save_path)
# model.config.save_pretrained(model_save_path)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}


In [ ]:

import random

index = random.randint(0 ,  len(df_train)-1)
org_eng = df_train['eng'][index]
org_yor = df_train['yor'][index]

a = model.generate(**tokenizer.prepare_seq2seq_batch(org_yor,return_tensors='pt').to('cuda'))
pred_eng = tokenizer.batch_decode(a)

pred_eng = str(pred_eng)
pred_eng = re.sub("<pad> ","",pred_eng)
pred_eng = re.sub("</s>","",pred_eng)


pred_eng = re.sub("'","",pred_eng)
pred_eng = pred_eng.replace("[", "")
pred_eng = pred_eng.replace("]", "")


# Print input Yoruba, original English, and predicted English
print("Input Yoruba: ", org_yor)
print("Original English: ", org_eng)
print("Predicted English: ", pred_eng)

Input Yoruba:  Ṣafihan awọn ibugbe irokuro ti yika nipasẹ didan pẹlu Iwọoorun
Original English:  Reveal fantasy realms surrounded by glowing with sunset
Predicted English:  Reveal fantasy realms surrounded by glowing with sunset


In [ ]:
import pandas as pd
import time
from tqdm import tqdm

# Read the test data
df_test = pd.read_csv('data/cleaned_test_data_yor_eng_300.csv')

# Initialize lists to store original Yoruba, original English, and predicted English sentences
original_yor_sentences = []
original_eng_sentences = []
predicted_eng_sentences = []

# Start timing
start_time = time.time()

# Use tqdm to visualize the progress
for index, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Predicting"):
    # Extract Yoruba sentence
    org_yor = row['yor']

    # Generate prediction
    inputs = tokenizer.prepare_seq2seq_batch(org_yor, return_tensors='pt').to('cuda')
    pred_ids = model.generate(**inputs)
    pred_eng = tokenizer.batch_decode(pred_ids)

    # Store original Yoruba, original English, and predicted English sentences
    original_yor_sentences.append(org_yor)
    original_eng_sentences.append(row['eng'])
    predicted_eng_sentences.append(pred_eng[0])  # Assuming only one prediction is generated

# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Create a new DataFrame to store original Yoruba, original English, and predicted English sentences
predictions_df = pd.DataFrame({
    'org_yor': original_yor_sentences,
    'org_eng': original_eng_sentences,
    'pred_eng': predicted_eng_sentences
})

# Print the elapsed time
print("Time taken for prediction: {:.2f} seconds".format(elapsed_time))




Predicting:   0%|          | 0/296 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
Predicting: 100%|██████████| 296/296 [00:48<00:00,  6.16it/s]

Time taken for prediction: 48.07 seconds


In [ ]:

predictions_df['pred_eng'] = predictions_df['pred_eng'].astype(str)
predictions_df['pred_eng'] = predictions_df['pred_eng'].apply(lambda x: re.sub("<pad> " , "" , x))

predictions_df['pred_eng'] = predictions_df['pred_eng'].apply(lambda x: re.sub("</s>" , "" , x))

predictions_df['pred_eng'] = predictions_df['pred_eng'].apply(lambda x: re.sub("'","",x))
predictions_df['pred_eng'] = predictions_df['pred_eng'].apply(lambda x: x.replace("[", ""))
predictions_df['pred_eng'] = predictions_df['pred_eng'].apply(lambda x: x.replace("]", ""))
predictions_df['pred_eng'] = predictions_df['pred_eng'].apply(lambda x: x.replace('"', ""))

# Optionally, you can save the predictions to a CSV file
predictions_df.to_csv('/content/drive/MyDrive/colab_notebooks/hackathon/benin_hackathon/test_predictions.csv', index=False)

# Print the first few rows of the predictions DataFrame
print(display(predictions_df))


,org_yor,org_eng,pred_eng
0,Wo aworan ode oni lakoko laarin iṣe,Watch modern art during in the midst of action,Watch modern art during in the midst of action
1,Ṣafihan iṣẹ ọna ode oni ti n ṣe ifihan ni ipin...,Show modern art featuring in serene isolation,Reveal modern art featuring in serene isolation
2,Wo awọn ilu ojo iwaju ti yika nipasẹ laaye pẹl...,View futuristic cities surrounded by alive wit...,Watch futuristic cities surrounded by alive wi...
3,Ni iriri awọn ayipada akoko nigba didan pẹlu I...,Experience seasonal changes during glowing wit...,Experience seasonal changes during glowing wit...
4,Awọn itọpa igbo fọtoyiya pẹlu wiwo ti laarin iṣe,Photograph forest trails with a view of in the...,Photograph forest trails with a view of in the...
...,...,...,...
291,Ṣe afihan awọn iyalẹnu ayaworan ni eti ti laay...,Show architectural wonders at the edge of aliv...,Show architectural wonders at the edge of aliv...
292,Ṣe apejuwe awọn agbegbe irokuro lodi si ẹhin t...,Depict fantasy realms against the backdrop of ...,Depict fantasy realms against the backdrop of ...
293,Ṣe afihan awọn ilu ọjọ iwaju ti n wo laaye pẹl...,Show futuristic cities overlooking alive with ...,Depict futuristic cities overlooking alive wit...
294,Yaworan oke seresere ni awọn eti ti awọn lãrin...,Capture mountain adventures at the edge of in ...,Capture mountain adventures at the edge of in ...


None


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
pred_data = pd.read_csv('/content/drive/MyDrive/colab_notebooks/hackathon/benin_hackathon/test_predictions.csv')

# Print the first few rows of the DataFrame
print(display(pred_data))

# Initialize a variable to count matching rows
num_matching_rows = 0

# Loop through each row of the DataFrame
for index, row in pred_data.iterrows():
    # Check if 'org_eng' matches 'pred_eng' for the current row
    org = row['org_eng']
    pred = row['pred_eng']
    print()
    if org == pred:
        num_matching_rows += 1
    else:

        print('org_test_eng - > ' , org)
        print('pred_test_eng - > ' , pred)



print("MATCHES :", num_matching_rows)

print("NO-MATCHES ", len(pred_data)- num_matching_rows)



,org_yor,org_eng,pred_eng
0,Wo aworan ode oni lakoko laarin iṣe,Watch modern art during in the midst of action,Watch modern art during in the midst of action
1,Ṣafihan iṣẹ ọna ode oni ti n ṣe ifihan ni ipin...,Show modern art featuring in serene isolation,Reveal modern art featuring in serene isolation
2,Wo awọn ilu ojo iwaju ti yika nipasẹ laaye pẹl...,View futuristic cities surrounded by alive wit...,Watch futuristic cities surrounded by alive wi...
3,Ni iriri awọn ayipada akoko nigba didan pẹlu I...,Experience seasonal changes during glowing wit...,Experience seasonal changes during glowing wit...
4,Awọn itọpa igbo fọtoyiya pẹlu wiwo ti laarin iṣe,Photograph forest trails with a view of in the...,Photograph forest trails with a view of in the...
...,...,...,...
291,Ṣe afihan awọn iyalẹnu ayaworan ni eti ti laay...,Show architectural wonders at the edge of aliv...,Show architectural wonders at the edge of aliv...
292,Ṣe apejuwe awọn agbegbe irokuro lodi si ẹhin t...,Depict fantasy realms against the backdrop of ...,Depict fantasy realms against the backdrop of ...
293,Ṣe afihan awọn ilu ọjọ iwaju ti n wo laaye pẹl...,Show futuristic cities overlooking alive with ...,Depict futuristic cities overlooking alive wit...
294,Yaworan oke seresere ni awọn eti ti awọn lãrin...,Capture mountain adventures at the edge of in ...,Capture mountain adventures at the edge of in ...


None


org_test_eng - >  Show modern art featuring in serene isolation
pred_test_eng - >  Reveal modern art featuring in serene isolation

org_test_eng - >  View futuristic cities surrounded by alive with color
pred_test_eng - >  Watch futuristic cities surrounded by alive with color





org_test_eng - >  Show time-lapse cityscapes with a view of bathed in moonlight
pred_test_eng - >  Reveal time-lapse cityscapes with a view of bathed in moonlight

org_test_eng - >  Present historical scenes during in serene isolation
pred_test_eng - >  Show historical scenes during in serene isolation




org_test_eng - >  Photograph desert sunsets showcasing glowing with sunset
pred_test_eng - >  Picture desert sunsets showcasing glowing with sunset




org_test_eng - >  Picture classical portraits during reflecting the morning sun
pred_test_eng - >  Photograph classical portraits during reflecting the morning sun

org_test_eng - >  Watch cultural ceremonies with a view of bathed in moonlight
pred_t